In [1]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')
import torch
from transformers import BertTokenizer, BertModel
import matplotlib.pyplot as plt
import pickle
import time
import numpy as np
# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
import logging
from scipy import stats
import random
from math import floor

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jafar\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import psycopg2
con = psycopg2.connect(database="postgres", user="postgres", password="Jafarsql", host="localhost", port="5432")
print("Database opened successfully")

cur = con.cursor()
q='''SELECT tweet FROM manager_world_tweets_ph2 
    Where 
    tweet_created_at like '%2019%';'''
cur.execute(q)
rows_mng = cur.fetchall()
# there is no repetative tweet
con.close()

print(len(rows_mng))

Database opened successfully
5983238


In [3]:
import psycopg2
con = psycopg2.connect(database="postgres", user="postgres", password="Jafarsql", host="localhost", port="5432")
print("Database opened successfully")

cur = con.cursor()
q='''SELECT tweet FROM ent_world_tweets_ph2 
    Where 
    tweet_created_at like '%2019%';'''
cur.execute(q)
rows_ent = cur.fetchall()
# there is no repetative tweet
con.close()

print(len(rows_ent))

Database opened successfully
5237437


In [4]:
from nltk.corpus import stopwords
cachedStopWords = stopwords.words("english")

from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
Stem=stemmer.stem

In [5]:
def cleaning (text):
    
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    #removing emoji
    text = emoji_pattern.sub(r' ', text)   

    #removeing http and https (URL)
    text = re.sub(r'(http://|https://)\S+', '', text)
    
    #removing www (URL)
    text=re.sub(r'www\.\S+', '', text)
    
    #removing targets
    text=re.sub('( |^)@\S+', '', text) 
    
    #removing tabs and lines
    text=re.sub('\t|\n', '', text)

    #removing some special charachter  
    text= re.sub("[\"\“\”\+\-\|\*\?\(\)\/\\\^\[\]\.{}_`;•«,@:~!=%&]+", ' ',text) # except ' ’
    
    #removing hashtag
    text=re.sub('#', '', text) 
    
    #removing numbers
    text=re.sub("(\d+)",' ',text)
#    print(text)
    #removing numbers (not attached)
#    text=re.sub(r"(^|\s)-?(\d+)\.?(\d*)(\s)", '', text)
#    text=re.sub("(^||\d+)\.(\d+) ",'',text)
#    text=re.sub("[0-9+]\.?(\d+)(\.)",' ',text)
#    text=re.sub("[0-9+]\.?(\d+)($)",' ',text)
#    print(text)
    
#    return text

#def lower_case (text):
    text = text.lower() 
#    print(text)
#    return text_lower

#def tokenization (text):
    text= nltk.word_tokenize(text)
#    print (text)
#    return text_tokens

#def removing_stopwords (text):
    #text_without_sw = [word.lower() for word in text if word.lower() not in stopwords.words()]
#    text = [word for word in text if word not in stopwords.words()]
    text = [word for word in text if word not in cachedStopWords]
#    print(text)
#    return text_without_sw

#def stemming (text):
    #stemmer = SnowballStemmer("english")
    #text_stems=stemmer.stem(tokens_without_sw)
#    text = [stemmer.stem(word) for word in text]
    text = [Stem(word) for word in text]

#    print(text)
    
    text=' '.join(text)
#    print(text)
#    print(text,'\n')   
    
    return text

In [6]:
mng_cleaned=[]
for tweet in rows_mng:
    ans=cleaning(tweet[0])
    if ans!=[]:
        mng_cleaned.append(ans)

In [7]:
ent_cleaned=[]
for tweet in rows_ent:
    ans=cleaning(tweet[0])
    if ans!=[]:
        ent_cleaned.append(ans)

In [8]:
#rows_ent[0][0]

In [9]:
ent_words=[]
for tweet in ent_cleaned:
    ent_words += tweet.split()

In [10]:
mng_words=[]
for tweet in mng_cleaned:
    mng_words += tweet.split()

For intra similarity, for example, for ent tweets we have

In [11]:
chi2=[]
p_value=[]

for kk in range(3):
    
    rang=range(len(ent_words))
    ind1=random.sample(rang,floor(len(ent_words)/2))
    ind2=[x for x in rang if x not in ind1]
    
    ent1_words_random=[]
    for ii in range(len(ind1)):
        ent1_words_random.append(ent_words[ind1[ii]])
        
    ent2_words_random=[]
    for ii in range(len(ind2)):
        ent2_words_random.append(ent_words[ind2[ii]])  
            
    all_words=ent_words

    d_ent1 = dict()
    for c in ent1_words_random:
        if c not in d_ent1:
            d_ent1[c] = 1
        else:
            d_ent1[c] = d_ent1[c] + 1

    d_ent1_new=dict(sorted(d_ent1.items(), key=lambda x: x[1], reverse=True))
    key_list_ent1=list(d_ent1_new.keys())
    values_list_ent1=list(d_ent1_new.values())
#    print(list(d_ent1_new.keys())[0:10])
#    print(list(d_ent1_new.values())[0:10]) 
#    print()
    
    d_ent2 = dict()
    for c in ent2_words_random:
        if c not in d_ent2:
            d_ent2[c] = 1
        else:
            d_ent2[c] = d_ent2[c] + 1
            
#    print(list(d_ent2_new.keys())[0:10])
#    print(list(d_ent2_new.values())[0:10])   
#    print()

    d_ent2_new=dict(sorted(d_ent2.items(), key=lambda x: x[1], reverse=True))
    key_list_ent2=list(d_ent2_new.keys())
    values_list_ent2=list(d_ent2_new.values())
#    print(list(d_all_new.keys())[0:10])
#    print(list(d_all_new.values())[0:10]) 
#    print()

    d_all = dict()
    for c in all_words:
        if c not in d_all:
            d_all[c] = 1
        else:
            d_all[c] = d_all[c] + 1

    d_all_new=dict(sorted(d_all.items(), key=lambda x: x[1], reverse=True))
    key_list_all=list(d_all_new.keys())
    values_list_all=list(d_all_new.values())
        

    frequent=key_list_all[0:10] # 
#    print(frequent)

    text1_freq_values=[]
    text2_freq_values=[]

    for ii in frequent:

        text1_freq_values.append(d_ent1_new[ii])
        text2_freq_values.append(d_ent2_new[ii])
        
        
#    print(text1_freq_values)  
#    print(text2_freq_values)  

    chi2_stat, p_val, dof, ex = stats.chi2_contingency([text1_freq_values,text2_freq_values])
    chi2.append(chi2_stat)
    p_value.append(p_val)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt



In [ ]:
CHI2_mean=np.mean(chi2)
CHI2_std=np.std(chi2)
P_VALUE_mean=np.mean(p_value)
P_VALUE_std=np.mean(p_value)

print('chi2:',chi2)
print('CHI2_mean:',CHI2_mean)
print('CHI2_std:',CHI2_std)
print('p_value:',p_value)
print('P_VALUE_mean:',P_VALUE_mean)
print('P_VALUE_std:',P_VALUE_std)

For example for one random sampling from the ent tweets and ctreating two subsets  we have these frequent words for the two subsets with this frequencies:

In [53]:
t0 = time.time()  
rang=range(10)  
ind1=random.sample(rang,floor(len(rang)/2))  
  
print( time.time() - t0)

0.0


In [54]:
a_set=set(rang)
ind1_set=set(ind1)
#ind1_set

In [55]:
t0 = time.time() 
ans1=a_set-ind1_set
ans1
print( time.time() - t0)

0.0


In [56]:
t0 = time.time() 
ans2=a_set.difference(ind1_set)
ans2
print( time.time() - t0)

0.0


In [57]:
ans1==ans2

True

In [58]:
ans1

{0, 2, 5, 6, 9}

In [61]:
aa=list(ans1)

In [63]:
aa[:floor(len(aa)/2)]

[0, 2]

In [64]:
aa[floor(len(aa)/2):]

[5, 6, 9]

In [65]:
rang=range(20)
slice1_ind=random.sample(rang,floor(20/4))
temp=list( set(rang) - set(slice1_ind)  ) # list(a.diference(b))
slice2_ind=random.sample(temp,floor(len(temp)/3))

In [67]:
slice1_ind

[1, 0, 7, 18, 15]

In [68]:
slice2_ind

[12, 14, 17, 8, 11]